In [3]:
import plotly
import plotly.graph_objs as go
from IPython.display import display, HTML
plotly.offline.init_notebook_mode(connected=True)
from loadbalanceRL import RAINMAN3
from loadbalanceRL.lib.algorithm.plot_dynamic import *

Rainman2's logging has been configured!


Using TensorFlow backend.


In [6]:
# Server profile: num_ues=120, APs=16, Scale=200.0, explore_radius=1
# If you want to change the num of ues or aps go to reset that in the server.py
# the EPISODES shows here is 100
from collections import OrderedDict
ALGORITHM_CONFIG = OrderedDict(
    EPISODES=3,
    ALPHA=0.2,
    GAMMA=0.9,
    EPSILON=0.3,
    EPSILON_DECAY=0.99,
    EPSILON_MIN=0.01,
    VERBOSE=True,
    L1_HIDDEN_UNITS=13,
    L2_HIDDEN_UNITS=13,
    L1_ACTIVATION='relu',
    L2_ACTIVATION='relu',
    LOSS_FUNCTION='mean_squared_error',
    OPTIMIZER='Adam',
)

CELLULAR_MODEL_CONFIG = OrderedDict(
    NAME='Cellular',
    TYPE='Dev',
    SERVER='0.0.0.0',
    SERVER_PORT='8000',
    VERBOSE=True,
)

In [7]:
RAINMAN3.algorithm_config = ALGORITHM_CONFIG
RAINMAN3.environment_config = CELLULAR_MODEL_CONFIG
result = RAINMAN3.run_experiment("Cellular", "Qlearning", "Naive")
print(result.BR_LIST)

2018-08-10 14:15:19,174 Mingyangs-MacBook-Pro.local Rainman2[54881] INFO Starting experiment!
2018-08-10 14:15:19,175 Mingyangs-MacBook-Pro.local Rainman2[54881] INFO Building Environment instance: Cellular
2018-08-10 14:15:19,176 Mingyangs-MacBook-Pro.local loadbalanceRL.lib.environment.cellular.base[54881] INFO Instantiating Cellular client: Dev
2018-08-10 14:15:19,178 Mingyangs-MacBook-Pro.local QCellularAgent[54881] INFO Configuration used for the QCellular Agent:
2018-08-10 14:15:19,178 Mingyangs-MacBook-Pro.local QCellularAgent[54881] INFO episodes: 3
2018-08-10 14:15:19,179 Mingyangs-MacBook-Pro.local QCellularAgent[54881] INFO alpha: 0.2
2018-08-10 14:15:19,180 Mingyangs-MacBook-Pro.local QCellularAgent[54881] INFO gamma: 0.9
2018-08-10 14:15:19,181 Mingyangs-MacBook-Pro.local QCellularAgent[54881] INFO epsilon: 0.3
2018-08-10 14:15:19,181 Mingyangs-MacBook-Pro.local QCellularAgent[54881] INFO epsilon_decay: 0.99
2018-08-10 14:15:19,182 Mingyangs-MacBook-Pro.local QCellularAgen

2018-08-10 14:15:20,905 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 59 is handed off from: 7 to : 11
2018-08-10 14:15:21,002 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 95 is handed off from: 6 to : 7
2018-08-10 14:15:21,032 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 103 is handed off from: 7 to : 11
2018-08-10 14:15:21,074 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 116 is handed off from: 10 to : 6
2018-08-10 14:15:21,121 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 131 is handed off from: 10 to : 7
2018-08-10 14:15:21,156 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 139 is handed off from: 11 to : 7
2018-08-10 14:15:21,173 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 142 is handed off from: 6 to : 11
2018-08-10 14:15:21,185 Mingyangs-MacBook-Pro.local CellularNetworkEnv[54881] INFO UE: 143 is handed off from: 2 to : 5
2018-08-10 14:15:21,213 Mingyangs-Ma

Module: run_experiment took: 2.0808529999999905s
[<BR {'br_id': 1, 'br_type': 'road', 'location': [585, 685, 0, 1600.0], 'direction': 'NS'}>, <BR {'br_id': 2, 'br_type': 'building', 'location': [1293, 1426, 214, 315], 'direction': 'random'}>, <BR {'br_id': 3, 'br_type': 'building', 'location': [1258, 1416, 1425, 1445], 'direction': 'random'}>, <BR {'br_id': 4, 'br_type': 'road', 'location': [1052, 1152, 0, 1600.0], 'direction': 'NS'}>]


[========================================================================] 100%


In [11]:
br_list = result.BR_LIST 
BR_locations=[]
BR_locations += [x.location for x in br_list]
Rec_dict=[]
for i in range(len(br_list)):
    Rec_dict.append({
            'type': 'rect',
            'x0': BR_locations[i][0],
            'y0': BR_locations[i][2],
            'x1': BR_locations[i][1],
            'y1': BR_locations[i][3],
            'line': {
                'color': 'rgba(128, 0, 128, 1)',
                'width': 2,
            },
            'fillcolor': 'rgba(128, 0, 128 0.1)'
          })
data, Frames_data, last_episode, layout_dict = plot_dynamic_data(result, Rec_dict, ALGORITHM_CONFIG['EPISODES'])


In [12]:
## Plot the APs and roads
ue_ap_list = result.UE_AP_LIST[0]
nodes_AP= [(0, 0)]
nodes_AP += [tuple(x.location) for x in ue_ap_list[0:16]]
nodes_color = []
nodes_color += ['orange'] * 16
n_x, n_y = zip(*nodes_AP[1:])
nodes_dict = dict(
        type='scatter',
        x=n_x,
        y=n_y,
        mode='markers',
        marker=dict(size=8, color=nodes_color)
    )

figure={"data":[nodes_dict],
        "layout": {'xaxis':{'range': [0, 1600], 'title': 'x'},
                   'yaxis':{'range': [0, 1600], 'title': 'y'},
                   'title':"UE and AP Distribution in the grid",
                   'height' : 1000,
                   'width' :1000,
                   'showlegend': False,
                   "shapes":Rec_dict, 
                  }
       }
plotly.offline.iplot(figure)



In [13]:
## Plot only the APs
ue_ap_list = result.UE_AP_LIST[0]
nodes_AP= [(0, 0)]
nodes_AP += [tuple(x.location) for x in ue_ap_list[0:16]]
nodes_color = []
nodes_color += ['orange'] * 16
n_x, n_y = zip(*nodes_AP[1:])
nodes_dict = dict(
        type='scatter',
        x=n_x,
        y=n_y,
        mode='markers',
        marker=dict(size=8, color=nodes_color)
    )

figure={"data":[nodes_dict],
        "layout": {'xaxis':{'range': [0, 1600], 'title': 'x'},
                   'yaxis':{'range': [0, 1600], 'title': 'y'},
                   'title':"UE and AP Distribution in the grid",
                   'height' : 1000,
                   'width' :1000,
                   'showlegend': False,
                   "shapes":Rec_dict, 
                  }
       }
plotly.offline.iplot(figure)




In [14]:
ue_ap_list = result.UE_AP_LIST[0]
nodes = nodes = [(0, 0)]
nodes += [tuple(x.location) for x in ue_ap_list]
nodes_color = []
print(nodes_color)
nodes_color += ['orange'] * 16
for ue in ue_ap_list[16:]:
    if ue.app == 2:
            # video UEs
        color = 'blue'
    else:
            # Web UEs
        color = 'magenta'
    nodes_color.append(color)
n_x, n_y = zip(*nodes[1:])
nodes_dict = dict(
    type='scatter',
    x=n_x,
    y=n_y,
    mode='markers',
    marker=dict(size=8, color=nodes_color)
    )

figure={"data":[nodes_dict],
        "layout": {'xaxis':{'range': [0, 1600], 'title': 'x'},
                   'yaxis':{'range': [0, 1600], 'title': 'y'},
                   'title':"UE and AP Distribution in the grid",
                   'height' : 1000,
                   'width' :1000,
                   'showlegend': False,
                   "shapes":Rec_dict, 
                  }
       }
plotly.offline.iplot(figure)



[]


In [15]:
#plot of ue and ap connection
ue_ap_list = result.UE_AP_LIST[0]
nodes = nodes = [(0, 0)]
nodes += [tuple(x.location) for x in ue_ap_list]
nodes_color = []
print(nodes_color)
nodes_color += ['orange'] * 16
for ue in ue_ap_list[16:]:
    if ue.app == 2:
            # video UEs
        color = 'blue'
    else:
            # Web UEs
        color = 'magenta'
    nodes_color.append(color)
n_x, n_y = zip(*nodes[1:])
nodes_dict = dict(
    type='scatter',
    x=n_x,
    y=n_y,
    mode='markers',
    marker=dict(size=8, color=nodes_color)
    )
edges = []
edge_color = []
for i, ue in enumerate(ue_ap_list[16:]):
    if ue.sla == 1:
        color = "green"
    else:
        color = "red"
    edges.append((ue.ap, i+17))
    edge_color.append(color)
edges_list = []
for k, e in enumerate(edges):
    info = dict(
        type='scatter',
        x=[nodes[e[0]][0], nodes[e[1]][0]],
        y=[nodes[e[0]][1], nodes[e[1]][1]],
        mode='lines',
        line=dict(width=2, color=edge_color[k])
    )
    edges_list.append(info)
data = edges_list + [nodes_dict]

figure={"data":data,
        "layout": {'xaxis':{'range': [0, 1600], 'title': 'x'},
                   'yaxis':{'range': [0, 1600], 'title': 'y'},
                   'title':"UE and AP Distribution in the grid",
                   'height' : 1000,
                   'width' :1000,
                   'showlegend': False,
                   "shapes":Rec_dict, 
                  }
       }
plotly.offline.iplot(figure)


[]


In [16]:

# print(data_0_episode["data"])
# layout_dict = dict(
#             xaxis={'range': [0, 1600], 'title': 'x'},
#             yaxis={'range': [0, 1600], 'title': 'y'},
#             title="UE and AP Distribution in the grid",
#             height=600,
#             width=800,
#             showlegend=False,
#             shapes=Rec_dict,
#             )
#layout = go.Layout(layout_dict
data_0_episode = Info_plot_dynamic_UE(result, 0)
data_values=data_0_episode["data"]  

figure={"data":data_values,
        "layout": {'xaxis':{'range': [0, 1600], 'title': 'x'},
                   'yaxis':{'range': [0, 1600], 'title': 'y'},
                   'title':"UE and AP Distribution in the grid",
                   'height' : 1000,
                   'width' :1000,
                   'showlegend': False,
                   "shapes":Rec_dict, 
                  }
       }
#plotly.offline.iplot(figure, image='png')


In [17]:
# Create rewards graph
rewards = go.Scatter(
    x = len(result.Rewards),
    y = result.Rewards,
    name='Rewards'
)

data = [rewards]

plotly.offline.iplot(
    {"data": data,
     "layout": {'xaxis':{'range': [0, 301], 'title': 'x'},
                   'yaxis':{'range': [100, 320], 'title': 'y'},
                   'title':"Rewards over time using Q-tabular",
                   'height' : 1000,
                   'width' :1000,
                   'showlegend': False,
                  }
    })

# Plot and embed in ipython notebook!
# plotly.offline.iplot(
#     {"data": data,
#      "layout": go.Layout(
#          height=500,
#          width=800,
#          title="Rewards over time",
#          xaxis=dict(
#              range=[0, 300]
#              title='Episodes',
#          ),
#          yaxis=dict(
#              title='Rewards',
#          ), 
#          showlegend=True)},
#     show_link=False,
#     link_text=None)




In [18]:
# Create handoff graph
handoffs = go.Scatter(
    x = len(result.Handoffs),
    y = result.Handoffs,
    name='Handoffs'
)

data = [handoffs]

# Plot and embed in ipython notebook!
plotly.offline.iplot(
    {"data": data,
     "layout": go.Layout(
         height=500,
         width=800,
         title='Handoffs over time',
         xaxis=dict(
             title='Episodes',
         ),
         yaxis=dict(
             title='Handoffs'
         ), 
         showlegend=True)},
    show_link=False,
    link_text=None)



In [19]:
# Create UE SLA graph
ue_sla = go.Scatter(
    x = len(result.Meets_SLA),
    y = result.Meets_SLA,
    name='UE SLA'
)

data = [ue_sla]

# Plot and embed in ipython notebook!
plotly.offline.iplot(
    {"data": data,
     "layout": {'xaxis':{'range': [0, 301], 'title': 'x'},
                   'yaxis':{'range': [80, 140], 'title': 'UE SLA'},
                   'title':"UE's SLA over episodes using Q-tabular",
                   'height' : 1000,
                   'width' :1000,
                   'showlegend': False,
                  }
    })